# Delta Lake Lab 
## Unit 4: CRUD Support

This lab is powered by Dataproc Serverless Spark.

In the previous unit, we -
1. Create an unpartitioned delta table
2. Created a partitioned delta table called loan_db.loans_by_state_delta
3. Studied the files created & layout in the datalake
4. Learned how to look at delta table details
5. Looked at history (there was not any)
6. Created a manifest file
7. Reviewed entries in the Hive metastore

In this unit, we will learn how to -
1. Delete a record and study the delta log
2. Insert a record and study the delta log
3. Update a record and study the delta log
4. Upsert and study the delta log

### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

23/12/02 23:38:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-diy-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-diy-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  11002190840


In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"

In [7]:
!gsutil ls -r $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-11002190840/delta-consumable/:
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet

gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/:
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000000.json

gs://dll-data-bucket-11002190840/delta-consumable/_symlink_format_manifest/:
gs://dll-data-bucket-11002190840/delta-consumable/_symlink_format_manifest/
gs://dll-data-bucket-11002190840/delta-consumable/_symlink_format_manifest/manifest


### 4. Delete support

In [8]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [9]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used
23/12/02 23:39:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----+
|addr_state|count|
+----------+-----+
|IA        |1    |
+----------+-----+



In [10]:
spark.sql("DELETE FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show(truncate=False)

+-----------------+
|num_affected_rows|
+-----------------+
|1                |
+-----------------+



In [11]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [12]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show(truncate=False)

+----------+-----+
|addr_state|count|
+----------+-----+
+----------+-----+



Lets look at the data lake:

In [13]:
# Note how the deleted created a json in the delta log directory
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/_delta_log/* 

gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000001.json


Lets look at the delta log:

In [14]:
# This is the original log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000000.json 

{"commitInfo":{"timestamp":1701559198463,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numOutputRows":"51","numOutputBytes":"978"},"engineInfo":"Apache-Spark/3.4.0 Delta-Lake/2.4.0","txnId":"ef45b4d8-1b6b-4e35-8d69-658760b727e5"}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"347aa570-3d50-4d3a-8aba-02dcf5b8bcee","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"addr_state\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"count\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1701559185896}}
{"add":{"path":"part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet","partitionValues":{},"size":978,"modificationTime":1701559194480,"dataChange":true,"stats":"{\"numRecords\":51,\"minValues\

In [15]:
# Note the delete in this log
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000001.json 

{"commitInfo":{"timestamp":1701560368300,"operation":"DELETE","operationParameters":{"predicate":"[\"(addr_state#361 = IA)\"]"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numRemovedBytes":"978","numCopiedRows":"50","numAddedChangeFiles":"0","executionTimeMs":"7057","numAddedFiles":"1","numDeletedRows":"1","scanTimeMs":"5699","numAddedBytes":"973","rewriteTimeMs":"1355"},"engineInfo":"Apache-Spark/3.4.0 Delta-Lake/2.4.0","txnId":"d8cc6548-51b4-4524-afa6-2eddb101be40"}}
{"remove":{"path":"part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet","deletionTimestamp":1701560368230,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":978}}
{"add":{"path":"part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet","partitionValues":{},"size":973,"modificationTime":1701560368094,"dataChange":true,"stats":"{\"numRecords\":50,\"minValues\":{\"addr_state\":\"AK\",\"count\":1},\"maxValu

### 5. Create (Insert) support

In [16]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [17]:
spark.sql("INSERT INTO loan_db.loans_by_state_delta VALUES ('IA',222222)")

DataFrame[]

In [18]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show(truncate=False)

+----------+------+
|addr_state|count |
+----------+------+
|IA        |222222|
+----------+------+



In [19]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [20]:
# Note how the insert created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-11002190840/delta-consumable/:
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-fe2f9136-6d38-4d3e-8090-bd41da49fe18-c000.snappy.parquet

gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/:
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000002.json

gs://dll-data-bucket-11002190840/delta-consumable/_symlink_format_manifest/:
gs://dll-data-bucket-11002190840/delta-consumable/_symlink_format_manifest/
gs://dll-data-bucket-11002190840/delta-consumable/_sy

In [21]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [22]:
# Lets check for the insert
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000002.json 

{"commitInfo":{"timestamp":1701560388412,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":1,"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"1","numOutputRows":"1","numOutputBytes":"725"},"engineInfo":"Apache-Spark/3.4.0 Delta-Lake/2.4.0","txnId":"5c356223-389e-45dc-8819-d623f260ad9d"}}
{"add":{"path":"part-00000-fe2f9136-6d38-4d3e-8090-bd41da49fe18-c000.snappy.parquet","partitionValues":{},"size":725,"modificationTime":1701560388335,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":222222},\"maxValues\":{\"addr_state\":\"IA\",\"count\":222222},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}


### 6. Update support

Lets update a record & see the changes in the delta log directory

In [23]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [24]:
spark.sql("UPDATE loan_db.loans_by_state_delta SET count = 11111 WHERE addr_state='IA'").show(truncate=False)

+-----------------+
|num_affected_rows|
+-----------------+
|1                |
+-----------------+



In [25]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show(truncate=False)

+----------+-----+
|addr_state|count|
+----------+-----+
|IA        |11111|
+----------+-----+



In [26]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [27]:
# Note how the update created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-11002190840/delta-consumable/:
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-af39e13d-1030-4881-965c-406884eb9420-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-fe2f9136-6d38-4d3e-8090-bd41da49fe18-c000.snappy.parquet

gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/:
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000001.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000002.json
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000003.json

g

In [28]:
# Lets check for the update
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000003.json 

{"commitInfo":{"timestamp":1701560408822,"operation":"UPDATE","operationParameters":{"predicate":"[\"(addr_state#1624 = IA)\"]"},"readVersion":2,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numRemovedBytes":"725","numCopiedRows":"0","numAddedChangeFiles":"0","executionTimeMs":"1597","scanTimeMs":"979","numAddedFiles":"1","numUpdatedRows":"1","numAddedBytes":"725","rewriteTimeMs":"616"},"engineInfo":"Apache-Spark/3.4.0 Delta-Lake/2.4.0","txnId":"c1be248e-6e17-4d27-913a-d3480e25ee17"}}
{"remove":{"path":"part-00000-fe2f9136-6d38-4d3e-8090-bd41da49fe18-c000.snappy.parquet","deletionTimestamp":1701560408820,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":725}}
{"add":{"path":"part-00000-af39e13d-1030-4881-965c-406884eb9420-c000.snappy.parquet","partitionValues":{},"size":725,"modificationTime":1701560408750,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":11111},\"maxVal

### 7. Upsert support

In [29]:
toBeMergedRows = [('IA', 555), ('CA', 12345), ('IN', 6666)]
toBeMergedColumns = ['addr_state', 'count']
toBeMergedDF = spark.createDataFrame(toBeMergedRows, toBeMergedColumns)
toBeMergedDF.createOrReplaceTempView("to_be_merged_table")
toBeMergedDF.orderBy("addr_state").show(3)

+----------+-----+
|addr_state|count|
+----------+-----+
|        CA|12345|
|        IA|  555|
|        IN| 6666|
+----------+-----+



In [30]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/delta_consumable/part* | wc -l 

CommandException: One or more URLs matched no objects.
0


In [31]:
spark.sql("DELETE FROM loan_db.loans_by_state_delta WHERE addr_state='IA'").show(truncate=False)

+-----------------+
|num_affected_rows|
+-----------------+
|1                |
+-----------------+



In [32]:
spark.sql("SELECT addr_state,count FROM loan_db.loans_by_state_delta WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show(truncate=False)

+----------+-----+
|addr_state|count|
+----------+-----+
|CA        |1    |
|IN        |1    |
+----------+-----+



In [33]:
mergeSQLStatement = "MERGE INTO loan_db.loans_by_state_delta as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * "

print(mergeSQLStatement)

MERGE INTO loan_db.loans_by_state_delta as d USING to_be_merged_table as m ON (d.addr_state = m.addr_state) WHEN MATCHED THEN UPDATE SET * WHEN NOT MATCHED THEN INSERT * 


In [34]:
spark.sql(mergeSQLStatement).show(truncate=False)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|3                |2               |0               |1                |
+-----------------+----------------+----------------+-----------------+



In [35]:
spark.sql("SELECT addr_state,count FROM loan_db.loans_by_state_delta WHERE addr_state in ('IA','CA','IN') ORDER BY addr_state").show(truncate=False)

+----------+-----+
|addr_state|count|
+----------+-----+
|CA        |12345|
|IA        |555  |
|IN        |6666 |
+----------+-----+



In [36]:
# Get the file count
!gsutil ls -r $DELTA_LAKE_DIR_ROOT/part* | wc -l

6


In [37]:
# Note how the update created a new parquet file and in the delta log, yet another json
!gsutil ls -r $DELTA_LAKE_DIR_ROOT 

gs://dll-data-bucket-11002190840/delta-consumable/:
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-18ffd7b0-964d-4f1e-a648-0892a1d4373d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-55d5ffb4-6b4a-4ead-8591-983dc631339f-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-6486345e-6c92-4216-b9ed-ffb413e41b4d-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-83d6b120-d178-4ac4-8c8e-f8d590f5f050-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-af39e13d-1030-4881-965c-406884eb9420-c000.snappy.parquet
gs://dll-data-bucket-11002190840/delta-consumable/part-00000-fe2f9136-6d38-4d3e-8090-bd41da49fe18-c000.snappy.parquet

gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/:
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/
gs://dll-data-bucket-11002190840/delta-consumable/_delta_log/00000000000000000000.json
gs://dll-data-bucket-110021

In [38]:
# Lets check for the upsert
!gsutil cat $DELTA_LAKE_DIR_ROOT/_delta_log/00000000000000000004.json 

{"commitInfo":{"timestamp":1701560433284,"operation":"DELETE","operationParameters":{"predicate":"[\"(addr_state#2311 = IA)\"]"},"readVersion":3,"isolationLevel":"Serializable","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"1","numRemovedBytes":"725","numCopiedRows":"0","numAddedChangeFiles":"0","executionTimeMs":"1814","numAddedFiles":"0","numDeletedRows":"1","scanTimeMs":"772","numAddedBytes":"0","rewriteTimeMs":"1042"},"engineInfo":"Apache-Spark/3.4.0 Delta-Lake/2.4.0","txnId":"5a96443a-661a-4149-9444-9ebbcf721608"}}
{"remove":{"path":"part-00000-af39e13d-1030-4881-965c-406884eb9420-c000.snappy.parquet","deletionTimestamp":1701560433283,"dataChange":true,"extendedFileMetadata":true,"partitionValues":{},"size":725}}


### 8. Review table history

In [39]:
deltaTable = DeltaTable.forPath(spark, DELTA_LAKE_DIR_ROOT)
fullHistoryDF = deltaTable.history()
fullHistoryDF.show(truncate=False)

+-------+-----------------------+------+--------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                                     

### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK